In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import dates
from matplotlib.colors import LogNorm
from sunpy.time import parse_time
from sunpy.coordinates import frames
import sunpy.map
from astropy.coordinates import SkyCoord
from astropy import units as u
import datetime
import pylab

In [2]:
# get all the data and adjust datetime etcs
all_data = pd.read_csv('concat_1996-2020.csv')
tt = pd.to_datetime(all_data['event_starttime'])
times = [(t - tt[0]).total_seconds() for t in tt]
all_data['times'] = times
all_data['tt'] = tt
all_data['area_arcsec'] = all_data['area_atdiskcenter']/(725**2)

In [3]:
# get all unique days over the solar cycle
tstart = parse_time(all_data['event_starttime'].min()).datetime
tfinal = parse_time(all_data['event_endtime'].max()).datetime
time_over = [tstart.strftime('%Y-%m-%dT%H:%M:%S')]
t0 = tstart
while t0 < tfinal:
    t0 = t0 + datetime.timedelta(days=1)
    time_over.append(t0.strftime('%Y-%m-%dT%H:%M:%S'))
all_data_test = all_data.set_index('tt')

In [4]:
# get number of spots etc as function of day
spots = []
no_ar = []
for i in range(len(time_over)):
    data_for_day = all_data[all_data['event_starttime'].isin([time_over[i]])]
    no_spots = data_for_day['ar_numspots'].sum()
    spots.append(no_spots)
    no_ar.append(len(data_for_day))
spots_ar = pd.Series(spots, index=parse_time(time_over).datetime)
ar_no = pd.Series(no_ar, index=parse_time(time_over).datetime)
ssn_tots = 10*ar_no+spots_ar

In [5]:
# get SSN data
def read_ssn():
    ssn = pd.read_csv("../SN_m_tot_V2.0.csv", names=['year', 'month', 'decimal_date', \
                                                  'ssn', 'ssn_dev', 'number_obs', 'indicator'], 
             delimiter=';')
    years = ssn['year'].values; months = ssn['month']
    tt = [datetime.datetime(ssn['year'][i], ssn['month'][i], 1) for i in range(len(ssn))]


    ssn['times'] = tt

    return ssn.set_index('times')
ssn = read_ssn()

In [6]:
sc23 = all_data_test.truncate('1996-09-01', '2009-01-01')
sc24 = all_data_test.truncate('2009-01-01', '2019-12-01')

sc25 = all_data_test.truncate('2019-12-01', '2020-08-21')

tstart = parse_time("2009-01-01").datetime
tfinal = parse_time("2020-08-21").datetime

In [7]:
# create an empty sunpy.map.Map
data_arr = np.zeros((1200, 1200))
coord = SkyCoord(0*u.arcsec, 0*u.arcsec, frame=frames.Helioprojective(observer='Earth', obstime=time_over[i]))
header = sunpy.map.make_fitswcs_header(data_arr, coord, scale=[2, 2]*u.arcsec/u.pix)
mapy = sunpy.map.Map(data_arr, header)

In [8]:
num_points = 100
lat_value = [-60, -45, -30, -15, 0, 15, 30, 45, 60]*u.deg
lon_value = [-60, -45, -30, -15, 0, 15, 30, 45, 60]*u.deg
lats = []
lons = []

for l in range(len(lat_value)):
    lat0 = SkyCoord(np.ones(num_points) * lat_value[l],
                np.linspace(-90, 90, num_points) * u.deg,
                frame=frames.HeliographicStonyhurst)
    lon0 = SkyCoord(np.linspace(-80, 80, num_points) * u.deg,
                np.ones(num_points) * lon_value[l], frame=frames.HeliographicStonyhurst)
    lat00 = lat0.transform_to(mapy.coordinate_frame)
    lon00 = lon0.transform_to(mapy.coordinate_frame)
    lats.append(lat00)
    lons.append(lon00)
coords_test = []
lat_value_plot = [-45, -30, -15, 0, 15, 30, 45]
for l in lat_value_plot:
    coords = SkyCoord(90*u.deg, l*u.deg, frame=frames.HeliographicStonyhurst).transform_to(mapy.coordinate_frame)
    coords_test.append(mapy.world_to_pixel(coords))

In [9]:
import matplotlib.colors as mcolors
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=-1):
    if n == -1:
        n = cmap.N
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
         'trunc({name},{a:.2f},{b:.2f})'.format(name=cmap.name, a=minval, b=maxval),
         cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [10]:
test_cmap = truncate_colormap(plt.get_cmap("Spectral"), minval=0, maxval=0.2)

In [19]:
cmap="Reds_r"
test_cmap = truncate_colormap(plt.get_cmap(cmap), minval=0, maxval=0.2)

def plot_24(cmap="Spectral", fname="test"):
    
    test_cmap = truncate_colormap(plt.get_cmap(cmap), minval=0, maxval=0.2)
    fig = plt.figure(figsize=(6, 10))

    # top plot
    ax1 = pylab.axes([0.2, 0.58, 0.60, 0.43], projection=mapy)
    # middle butterfly
    ax2 = pylab.axes([0.1, 0.305, 0.88, 0.33])
    # bottom ssn plot
    ax3 = pylab.axes([0.1, 0.05, 0.88, 0.25], sharex=ax2)

    sizey24 = 10*sc24['area_atdiskcenter']/sc24['area_atdiskcenter'].mean()
    sizey24 = 5*sc24['ar_numspots']

    sizey25 = 10*sc25['area_atdiskcenter']/sc25['area_atdiskcenter'].mean()
    sizey25 = 5*sc25['ar_numspots']

    ##### FIRST SOLAR CYCLE 24 ########
    # circle1 = plt.Circle((0, 0), 960, color='k', fill=False)

    mapy.plot(axes=ax1, alpha=0.0)
    mapy.draw_limb(color='k', lw=0.8)
    #mapy.draw_grid(color='w', lw=0.5)
    ax1.scatter((sc24['hpc_x'].values*u.arcsec).to(u.deg), 
                 (sc24['hpc_y'].values*u.arcsec).to(u.deg), 
                 s=sizey24, edgecolor='k', lw=0.1,
                 c=sc24['times'], alpha=0.5,
                transform=ax1.get_transform('world'), cmap=cmap)

    ax1.scatter((sc25['hpc_x'].values*u.arcsec).to(u.deg), 
                 (sc25['hpc_y'].values*u.arcsec).to(u.deg), 
                 s=sizey25, edgecolor='k', lw=0.1,
                 c=sc25['times'], alpha=0.5,
                transform=ax1.get_transform('world'), cmap=test_cmap)

    ax1.set_xlabel('X (arcsec)')
    ax1.set_ylabel(' ')
    ax1.tick_params(which='both',direction='in')
    ax1.tick_params(axis='y', labelleft=False)
    ax1.set_title('Solar Cycle 24', pad=-1)#.set_position([.5, 1])
    ax1.set_axis_off()
    for l in range(len(lats)):
        ax1.plot_coord(lats[l], color='k', lw=0.2)
        ax1.plot_coord(lons[l], color='k', lw=0.2)

    for c in range(len(coords_test)):
        ax1.text(coords_test[c].x.value+40, coords_test[c].y.value, str(lat_value_plot[c])+'$^\circ$')


    ### ax2
    ax2.scatter(sc24.index, sc24['hgc_y'], 
                alpha=0.5, c=sc24['times'], 
                s=sizey24/2,
                #s=100*all_data['ar_numspots']/all_data['ar_numspots'].mean(),
                cmap=cmap, edgecolor='k', lw=0.1)
    ax2.scatter(sc25.index, sc25['hgc_y'], 
                alpha=0.5, c=sc25['times'], 
                s=sizey25/2,
                #s=100*all_data['ar_numspots']/all_data['ar_numspots'].mean(),
                cmap=test_cmap, edgecolor='k', lw=0.1)

    ax2.set_ylim(-45, 45)
    ax2.set_ylabel('Latitude ($^\circ$)')
    ax2.tick_params(which='both',direction='in', labelbottom=False)
    ax2.xaxis.set_major_locator(dates.YearLocator(4))
    ax2.xaxis.set_minor_locator(dates.YearLocator(1))  
    ax2.xaxis.set_major_formatter(dates.DateFormatter('%Y'))  

    ## ax3

    ##### SUNSPOT NUMBER AS A FUNCTION OF TIME ########
    ax3.plot(ssn_tots.resample('30D').mean().truncate(tstart, tfinal), color='k')
    ax3.set_ylabel('Sunspot number')
    ax3.set_xlabel('Time')
    plt.tight_layout()
    ax3.set_xlim(tstart, tfinal)
    ax3.tick_params(which='both',direction='in')

    ax2.axvline("2019-12-01", ls="dashed", color="grey", lw=0.5)
    ax3.axvline("2019-12-01", ls="dashed", color="grey", lw=0.5)


    plt.savefig("test_{:s}.jpeg".format(fname), dpi=200)
    plt.close()

In [20]:
plot_24(fname="testy")

/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [12]:
plot_24(cmap="Spectral", fname="spectral")
plot_24(cmap="Reds_r", fname="Reds_r")
plot_24(cmap="Blues_r", fname="Blues_r")
plot_24(cmap="magma", fname="magma")
plot_24(cmap="plasma", fname="plasma")

/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [27]:
cmap="Reds_r"
test_cmap = truncate_colormap(plt.get_cmap(cmap), minval=0, maxval=0.2)

def plot_24_small(cmap="Spectral", fname="test"):
    
    test_cmap = truncate_colormap(plt.get_cmap(cmap), minval=0, maxval=0.2)
    fig = plt.figure(figsize=(6, 8))

    # top plot
    ax1 = pylab.axes([0.2, 0.5, 0.60, 0.43], projection=mapy)
    # middle butterfly
    ax2 = pylab.axes([0.1, 0.05, 0.88, 0.45])
    # bottom ssn plot
   # ax3 = pylab.axes([0.1, 0.05, 0.88, 0.25], sharex=ax2)

    sizey24 = 10*sc24['area_atdiskcenter']/sc24['area_atdiskcenter'].mean()
    sizey24 = 5*sc24['ar_numspots']

    sizey25 = 10*sc25['area_atdiskcenter']/sc25['area_atdiskcenter'].mean()
    sizey25 = 5*sc25['ar_numspots']

    ##### FIRST SOLAR CYCLE 24 ########
    # circle1 = plt.Circle((0, 0), 960, color='k', fill=False)

    mapy.plot(axes=ax1, alpha=0.0)
    mapy.draw_limb(color='k', lw=0.8)
    #mapy.draw_grid(color='w', lw=0.5)
    ax1.scatter((sc24['hpc_x'].values*u.arcsec).to(u.deg), 
                 (sc24['hpc_y'].values*u.arcsec).to(u.deg), 
                 s=sizey24, edgecolor='k', lw=0.1,
                 c=sc24['times'], alpha=0.5,
                transform=ax1.get_transform('world'), cmap=cmap)

    ax1.scatter((sc25['hpc_x'].values*u.arcsec).to(u.deg), 
                 (sc25['hpc_y'].values*u.arcsec).to(u.deg), 
                 s=sizey25, edgecolor='k', lw=0.1,
                 c=sc25['times'], alpha=0.5,
                transform=ax1.get_transform('world'), cmap=test_cmap)

    ax1.set_xlabel('X (arcsec)')
    ax1.set_ylabel(' ')
    ax1.tick_params(which='both',direction='in')
    ax1.tick_params(axis='y', labelleft=False)
    ax1.set_title('Solar Cycle 24', pad=-1)#.set_position([.5, 1])
    ax1.set_axis_off()
    for l in range(len(lats)):
        ax1.plot_coord(lats[l], color='k', lw=0.2)
        ax1.plot_coord(lons[l], color='k', lw=0.2)

    for c in range(len(coords_test)):
        ax1.text(coords_test[c].x.value+40, coords_test[c].y.value, str(lat_value_plot[c])+'$^\circ$')


    ### ax2
    ax2.scatter(sc24.index, sc24['hgc_y'], 
                alpha=0.5, c=sc24['times'], 
                s=sizey24/2,
                #s=100*all_data['ar_numspots']/all_data['ar_numspots'].mean(),
                cmap=cmap, edgecolor='k', lw=0.1)
    ax2.scatter(sc25.index, sc25['hgc_y'], 
                alpha=0.5, c=sc25['times'], 
                s=sizey25/2,
                #s=100*all_data['ar_numspots']/all_data['ar_numspots'].mean(),
                cmap=test_cmap, edgecolor='k', lw=0.1)

    ax2.set_ylim(-45, 45)
    ax2.set_ylabel('Latitude ($^\circ$)')
    ax2.tick_params(which='both',direction='in')#, labelbottom=False)
    ax2.axvline("2019-12-01", ls="dashed", color="grey", lw=0.5)
    ax2.set_xlabel('Time')
    ax2.set_xlim(tstart, tfinal)
    
    ax2.xaxis.set_major_locator(dates.YearLocator(2))
    ax2.xaxis.set_major_formatter(dates.DateFormatter('%Y'))
    ax2.xaxis.set_minor_locator(dates.YearLocator(1))  
     

    plt.tight_layout()
    plt.savefig("test2_{:s}.jpeg".format(fname), dpi=200)
    plt.close()

In [30]:
plot_24_small(cmap="Spectral", fname="spectral")
plot_24_small(cmap="Reds_r", fname="Reds_r")
plot_24_small(cmap="Blues_r", fname="Blues_r")
plot_24_small(cmap="magma", fname="magma")
plot_24_small(cmap="plasma", fname="plasma")

/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
/Users/laurahayes/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
